In [ ]:
#from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if kaggle.json is stored in Google Drive

Mounted at /content/drive


In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
#!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log       # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 22122shop    # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log              # download competition dataset as a zip file
!unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show           # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 22122shop
100% 9.98M/9.98M [00:00<00:00, 73.8MB/s]
Using competition: 22122shop
 teamId  teamName                                             submissionDate       score    
-------  ---------------------------------------------------  -------------------  -------  
8249741  IB-Valhalla- Adshead,Lukianov,Odnakov                2022-03-06 18:47:06  0.99140  
8857973  Samuel Nathanson                                     2022-07-09 00:34:54  0.98832  
8853383  Robbie Cahill                                        2022-07-08 04:10:49  0.98008  
8250279  JF-BigSpenders-Merran,Hogge                          2022-03-06 22:07:24  0.97396  
8228358  JA-Shopaholics-Brodsky,Rumman,Givre                  2022-03-06 19:29:55  0.97264  
8205432  JE-TBD-Mendola,Natali                                2022-03-07 04:27:19  0.97152  
8235054  JC-TheCashCows-Hernandez-Khan                        2022-03-06 04:40:33  0.96092

In [ ]:
!pip -q install -U tensorflow_addons > log

In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all" 
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, tensorflow_addons as tfa
from sklearn.preprocessing import PolynomialFeatures
import tensorflow as tf, tensorflow.keras as keras
from keras.layers import Flatten, Dense
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('max_columns', 20, 'precision', 2, 'display.max_rows', 4)

CPU times: user 3.69 s, sys: 453 ms, total: 4.14 s
Wall time: 4.5 s


In [ ]:
df = pd.read_csv('XY_Shop.csv'); df.head()

,Adm,AdmDur,Inf,InfDur,Prd,PrdDur,BncRt,ExtRt,PgVal,SpclDay,Mo,OS,Bsr,Rgn,TfcTp,VstTp,Wkd,Rev
0,0,0.00,0,0.0,18,132.99,3.82e-02,0.05,0.00,0.0,4,3,1,1,2,0,1,NaN
1,1,0.00,0,0.0,37,1150.20,1.25e-03,0.03,0.00,0.0,11,2,2,4,2,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,3,263.68,0,0.0,24,749.14,4.47e-03,0.02,14.58,0.0,7,2,2,2,1,0,1,NaN
4,0,0.00,0,0.0,3,136.41,0.00e+00,0.07,0.00,0.0,12,1,1,3,3,1,0,NaN


In [ ]:
vX = df.query('Rev!=Rev').drop('Rev', axis=1)  # slice a test sample
tXY = df.query('Rev==Rev')                     # slice training sample
tX, tY = tXY.drop('Rev', axis=1), tXY.Rev      # split into training I/O
NumFeatures = list(tX.select_dtypes(np.number).columns)

In [ ]:
#tmr = Timer()

<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**
 
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc. 
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

1. Observing the eda, it is apparent that the target class has a severe class imbalance. There are almost 3x the amount of the 0 class compared to that of the 1 class. Additionally observing the eda, some features such as "browser" and "OS" may not contribute much towards the dataset and can possibly dropped.


2. The effectiveness of these elements were evaluated through the resulting f1 score obtained after training the model. The scores were checked after each time an element was added or removed to check their effectiveness on raising the f1 score or training/validation accuracy. Additionally, observing the live scores get printed on screen from fitting the model to the data allowed us to quickly observe if a certain element we were testing caused the model to severely degrade. 


3. One of our first attempts was just to use the floating point columns how the base starter code had. This however, never brought our score above 82%. We settled on using the full feature set which allowed us to obtain higher scores. We noticed that there were some features/classes that were imbalanced and used random oversampling and SMOTE to generate new synthetic data in attempts to create more balance in the dataset. The result of using these methods on the dataset caused the validation accuracy to stall around 65% throughout the training iterations. We then tried to create more features through polynomial features at a degree of 2-3. We noticed that there were marginal increases in accuracy if any and that it only contributed to more compute time. Using StandardScaler provided the greatest training/validation accuracy at almost 90% however when tested with the f1 score, significantly overfitted and resulted in scores around 77%. Using PCA we found to give us the greatest increase in accuracy and f1 scores among the other preprocessing steps mentioned. It is known to use standardization before PCA however, when we tried this, it resulted in poor performance. 

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful. 

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

1. It is well known that deeper networks provide for better results. We saw this almost immediately experimenting with the effects of adding more layers to the base kaggle notebook and observing its effects on the accuracy obtained. Also experimenting with different activiation and optimizers such as "selu/relu" and "RMSprop/Adam" and their resulting accuracies on the model. 


2. Like Task 1, The effectiveness of these elements were evaluated through the resulting f1 score obtained after training the model. The scores were checked after each time an element was added or removed to check their effectiveness on raising the f1 score or training/validation accuracy. Additionally, observing the live scores get printed on screen from fitting the model to the data allowed us to quickly observe if a certain element we were testing caused the model to severely degrade.


3. We tried changing the amount of Dense layers as well as the number of units within each Dense layer. We saw that increasing the amount of Dense layers improves the accuracies obtained up to a certain point before it begins to plateau. Likewise, increasing the size of the units generally improved the accuracy but increasing the units to larger units caused a longer training duration. Another parameter we experimented with and saw some success with was changing the batch size and learning rate. We saw that increasing the batch size while decreasing the learning rate created a decent tradeoff as increasing the batch size causes a smaller amount of data to be processed per epoch, improving compute time and decreasing the learning causes slower convergence. Changing the number of epochs to something greater than 50 allowed us to obtain scores higher than 82 while still maintaining the 60s window. 

## Other Models and Their Resulting Accuracies

## EDA

Class imbalance with the target variable

Model 1

In [ ]:
# vX = df.query('Rev!=Rev').drop('Rev', axis=1)  # slice a test sample
# tXY = df.query('Rev==Rev')                     # slice training sample
# tX, tY = tXY.drop('Rev', axis=1), tXY.Rev      # split into training I/O
# NumFeatures = list(tX.select_dtypes(include='float').columns)
# print('Numeric features: ', NumFeatures)       # numeric/quantitative feature names

# train_data = tX.copy()


# train_data = train_data.drop(columns = ['Bsr','Rgn','TfcTp'],axis=1)
# train_data


# from sklearn import preprocessing
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler().fit(train_data.values)

# scaledf = scaler.transform(train_data.values) #training dataset
# scaled_training = pd.DataFrame(scaledf, index=train_data.index, columns=train_data.columns)
# scaled_training


# tX_in = scaled_training
# tY_in = tY

# m = keras.Sequential([
#     keras.layers.Dense(70, activation=tf.nn.relu,
#                      input_shape=[tX_in.shape[1]]),
#     #keras.layers.Dropout(0.25),
#     keras.layers.Dense(70, activation=tf.nn.relu),
#     #keras.layers.Dropout(0.25),
#     keras.layers.Dense(70, activation=tf.nn.relu),
#     keras.layers.Dense(70, activation=tf.nn.relu),
#     keras.layers.Dense(70, activation=tf.nn.relu),
# #     keras.layers.Dense(70, activation=tf.nn.relu),
#     #keras.layers.Dropout(0.25),
# #     keras.layers.Flatten(),
# #     keras.layers.Dense(10, activation=tf.nn.relu),
#     keras.layers.Flatten(),
#     keras.layers.Dense(10, activation=tf.nn.relu),
#     keras.layers.Dense(10, activation=tf.nn.relu),

#     keras.layers.Dense(1, activation= "sigmoid")
#   ])

# m.summary()
# m.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
# hist = m.fit(tX_in, tY_in, epochs=10, validation_split=0.3, batch_size=32)

accuracy: 0.9091 - val_loss: 0.2453 - val_accuracy: 0.9065

Model 2

In [ ]:
# vX = df.query('Rev!=Rev').drop('Rev', axis=1)  # slice a test sample
# tXY = df.query('Rev==Rev')                     # slice training sample
# tX, tY = tXY.drop('Rev', axis=1), tXY.Rev      # split into training I/O
# NumFeatures = list(tX.select_dtypes(np.number).columns)
# print('Numeric features: ', NumFeatures)     

# train_data = tX.copy()
# train_data = train_data.drop(columns = ['Inf','PgVal'],axis=1)

# from sklearn import preprocessing
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler().fit(train_data.values)

# scaledf = scaler.transform(train_data.values) #training dataset
# scaled_training = pd.DataFrame(scaledf, index=train_data.index, columns=train_data.columns)
# scaled_training

# tX_in = scaled_training
# tY_in = tY_in

# m = keras.Sequential([
#     keras.layers.Dense(70, activation=tf.nn.relu,
#                      input_shape=[tX_in.shape[1]]),
#     keras.layers.Dropout(0.25),
#     keras.layers.Dense(70, activation=tf.nn.relu),
#     keras.layers.Dropout(0.25),
#     keras.layers.Dense(70, activation=tf.nn.relu),
#     keras.layers.Dense(70, activation=tf.nn.relu),
#     keras.layers.Dense(70, activation=tf.nn.relu),
#     keras.layers.Dropout(0.25),
# #     keras.layers.Flatten(),
# #     keras.layers.Dense(10, activation=tf.nn.relu),
#     keras.layers.Flatten(),
#     keras.layers.Dense(10, activation=tf.nn.relu),
#     keras.layers.Dense(10, activation=tf.nn.relu),
#     keras.layers.Dense(10, activation=tf.nn.relu),
#     keras.layers.Dense(1, activation= "sigmoid")
#   ])

# m.summary()
# m.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
# hist = m.fit(tX_in, tY_in, epochs=20, validation_split=0.3, batch_size=32)



loss: 0.1885 - accuracy: 0.9210 - val_loss: 0.1994 - val_accuracy: 0.9199

Model 3

In [ ]:
# vX = df.query('Rev!=Rev').drop('Rev', axis=1)  # slice a test sample
# tXY = df.query('Rev==Rev')                     # slice training sample
# tX, tY = tXY.drop('Rev', axis=1), tXY.Rev      # split into training I/O
# NumFeatures = list(tX.select_dtypes(include='float').columns)
# print('Numeric features: ', NumFeatures)   

# train_data = tX.copy()

# from sklearn import preprocessing
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler().fit(train_data.values)

# scaledf = scaler.transform(train_data.values) #training dataset
# scaled_training = pd.DataFrame(scaledf, index=train_data.index, columns=train_data.columns)
# scaled_training


# from sklearn.decomposition import PCA
# pca = PCA(10)
# pca.fit(scaled_training)
# original_variance = pca.explained_variance_ratio_

# pca_data = pca.transform(scaled_training)

# tX_in = pca_data
# tY_in = tY

# m = keras.Sequential([ keras.layers.Dense(60, activation=tf.nn.relu, input_shape=[tX_in.shape[1]]), 
                                          
#   #keras.layers.Dropout(0.25), 
#   keras.layers.Dense(60, activation=tf.nn.relu, input_shape=[tX_in.shape[1]]), 
                                          
#   #keras.layers.Dropout(0.25), 
# keras.layers.Dense(60, activation=tf.nn.relu, input_shape=[tX_in.shape[1]]), 
# keras.layers.Dense(32,activation=tf.nn.relu), 
# #
# keras.layers.Dense(16,activation=tf.nn.relu), #kernel_regularizer=keras.regularizers.l2(0.1), ), 
# keras.layers.Dense(1, activation= "sigmoid") ])


# m.summary()
# m.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
# hist = m.fit(tX_in, tY_in, epochs=10, validation_split=0.3, batch_size=32)


loss: 0.2573 - accuracy: 0.9000 - val_loss: 0.2629 - val_accuracy: 0.8982

## Final Model

In [ ]:
train_data = tX.copy()

In [ ]:
train_data.shape

(450000, 17)

In [ ]:
tX_in = train_data[NumFeatures][:50000]
tY_in = tXY.Rev[:50000]

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(14)
pca.fit(tX_in)
original_variance = pca.explained_variance_ratio_
pca_data = pca.transform(tX_in)

PCA(n_components=14)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import regularizers
import tensorflow as tf, tensorflow.keras as keras
from keras.layers import Flatten, Dense
from tensorflow.keras.layers import BatchNormalization

In [ ]:
tf.random.set_seed(69)   # always seed your experiments
Init = tf.keras.initializers.RandomNormal(seed=69)

In [ ]:
tX_in = pca_data
tY_in = tY

m = keras.Sequential([
    keras.layers.Dense(90, activation=tf.nn.selu,
                     input_shape=[tX_in.shape[1]]),
    
    keras.layers.Dense(90, activation=tf.nn.selu),
    keras.layers.Dense(90, activation=tf.nn.selu),
    # keras.layers.Dense(90, activation=tf.nn.selu),
    # keras.layers.Dense(90, activation=tf.nn.selu),
    # keras.layers.Dense(90, activation=tf.nn.selu),
    # keras.layers.Dense(90, activation=tf.nn.selu),
    # keras.layers.Dense(90, activation=tf.nn.selu),
    
    keras.layers.Flatten(),
    keras.layers.Dense(90, activation=tf.nn.selu),
    keras.layers.Dense(90, activation=tf.nn.selu),# step size = 10
    keras.layers.Dense(32, activation=tf.nn.selu),# step size = 6

    keras.layers.Dense(1, activation= "sigmoid")
  ])


#### 
# NOTE FOR CARLOS AND NAVEEN ####
####
### The code below got 84% on kaggle and is shorter runtime than the code above in google collab. Im not sure why its taking 5 mins on this collab 
### notebook, when it only took 53 seconds on my computer to run inside a juypter notebook. Try playing with the layers and the number of epochs to adjust the time.



# tX_in = pca_data
# tY_in = tY

# m = keras.models.Sequential([
#     Flatten(input_shape=[tX_in.shape[1]], name='input'),
#     Dense(256, activation="selu", kernel_initializer=Init),
    
#     Dense(256, activation="selu", kernel_initializer=Init),

# #     Dropout(0.3),
# #     BatchNormalization(),
#     Dense(128, activation="selu", kernel_initializer=Init),
# #     Dropout(0.3),
# #     BatchNormalization(),
#     Dense(32, activation="selu", kernel_initializer=Init),

# #     Dropout(0.3),
# #     BatchNormalization(),
#     Dense(1, activation='sigmoid', kernel_initializer=Init, name='output')])

#m.summary()
m.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])
hist = m.fit(tX_in, tY_in, epochs=100, validation_split=0.3, batch_size=64)


Epoch 1/100
547/547 [==============================] - 4s 5ms/step - loss: 0.6871 - accuracy: 0.7750 - val_loss: 0.5705 - val_accuracy: 0.7907
Epoch 2/100
547/547 [==============================] - 3s 5ms/step - loss: 0.5193 - accuracy: 0.7939 - val_loss: 0.4760 - val_accuracy: 0.8020
Epoch 3/100
547/547 [==============================] - 3s 5ms/step - loss: 0.5074 - accuracy: 0.8008 - val_loss: 0.5175 - val_accuracy: 0.7828
Epoch 4/100
547/547 [==============================] - 2s 4ms/step - loss: 0.4991 - accuracy: 0.8030 - val_loss: 0.6284 - val_accuracy: 0.8058
Epoch 5/100
547/547 [==============================] - 3s 5ms/step - loss: 0.4899 - accuracy: 0.8052 - val_loss: 0.5343 - val_accuracy: 0.8059
Epoch 6/100
547/547 [==============================] - 2s 4ms/step - loss: 0.4844 - accuracy: 0.8073 - val_loss: 0.4525 - val_accuracy: 0.8109
Epoch 7/100
547/547 [==============================] - 3s 5ms/step - loss: 0.4709 - accuracy: 0.8097 - val_loss: 0.4733 - val_accuracy: 0.8074

In [ ]:
## uncomment to see f1 score

# pca_test_data = pca.transform(tXY[NumFeatures])
# y_test = tXY.Rev[50000:]
# y_pred = pd.DataFrame(m.predict(pca_test_data[50000:]))

# y_pred = y_pred.round(0).astype(int)

# from sklearn.metrics import f1_score
# f1_score(y_test, y_pred, average='micro')

0.84715

In [ ]:
pca_test_data = pca.transform(vX)

In [ ]:
pY = pd.DataFrame(m.predict(pca_test_data), index=vX.index+1, columns=['Rev'])
pY.index.names = ['id']
ToCSV(pY.round(0).astype(int), 'batch_size_epoch_100')
tXY.Rev.value_counts()/len(tXY)     

0.0    0.78
1.0    0.22
Name: Rev, dtype: float64

# **References:**

1. Remember to cite your sources!


<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
#tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

## 💡**Starter Ideas**

**Model**
1. Tune model hyperparameters, batch size, optimizer, NN layers

**Features**
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Incorporate categorical features (appropriately encoded)
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?

**Training observations**
1. Try clustering methods to remove similar observations. You may also try dimension reduction methods (eg. PCA) on the transposed data matrix (if it has scaled numeric features).
1. Look for and deal with outliers or influential points in the training set
1. Deal with **imbalanced sample**: oversample smaller class, or undersample larger class, or provide observation weights or provide class weights, or seek a suitable loss function
1. Investigate distributions of features. Any missing values? Any zero values?

**Predictions**
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest misclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.

**EDA and Domain Expertise**
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Learn about the domain: how should output relate to features? How do month or weekend impact users' buying activity?
  1. User Agent [&#127910;](https://www.youtube.com/results?search_query=user+agent+browser), Google Analytics [&#127910;](https://www.youtube.com/results?search_query=google+analytics), tracking online shopping intent [&#127910;](https://www.youtube.com/results?search_query=tacking+online+shopping+intent), [📄](https://scholar.google.com/scholar?q=tracking+online+shopping+intent)